In [1]:
import pandas as pd
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [2]:
import os
DATA_DIR = os.getcwd() + "/data"

In [3]:
tables = pd.read_html("https://en.wikipedia.org/wiki/UEFA_Euro_2024")

In [3]:
len(tables)

73

In [4]:
qf_teams_df = tables[4]


In [5]:
team_tables = {}
general_team_url = "https://en.wikipedia.org/wiki/toreplace_national_football_team"
for team in qf_teams_df["Team"]:
    print(team)
    sep = "["
    if sep in team:
        team = team.split(sep, 1)[0]
    team = team.replace(" ", "_")
    team_url = general_team_url.replace("toreplace", team)
    team_tables[team] = team_url

Germany[B]
Belgium
France
Portugal
Scotland
Spain
Turkey
Austria
England
Hungary
Slovakia[C]
Albania
Denmark
Netherlands
Romania
Switzerland
Serbia[D]
Czech Republic[C]
Italy
Slovenia
Croatia
Georgia
Ukraine
Poland


In [70]:
germany_info = pd.read_html(team_tables["Germany"], match="Player")

In [73]:
germany_df = germany_info[0]


In [66]:
belgium_info = pd.read_html(team_tables["Belgium"], match="Player")

In [89]:
belgium_df = belgium_info[0]
belgium_df.head()


,No.,Pos.,Player,Date of birth (age),Caps,Goals,Club
0,12.0,GK,Koen Casteels,(age 31),8.0,0.0,VfL Wolfsburg
1,13.0,GK,Matz Sels,(age 32),6.0,0.0,Nottingham Forest
2,1.0,GK,Arnaud Bodart,(age 26),0.0,0.0,Standard Liège
3,24.0,GK,Thomas Kaminski,(age 31),0.0,0.0,Luton Town
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
player_dfs = []

for team, url in team_tables.items():
    team_squad_info = pd.read_html(url, match="Player")
    tmp_df = team_squad_info[0]
    tmp_df["Goals"] = tmp_df["Goals"]
    tmp_df["Caps"] = tmp_df["Caps"]
    if "_" in team:
        team = team.replace("_", " ")
    tmp_df["Team"] = team
    player_dfs.append(tmp_df)


In [14]:
merged_df = pd.concat(player_dfs, ignore_index=True)
merged_df.head()

,No.,Pos.,Player,Date of birth (age),Caps,Goals,Club,Team
0,12.0,GK,Bernd Leno,(age 32),9.0,0.0,Fulham,Germany
1,22.0,GK,Marc-André ter Stegen,(age 31),40.0,0.0,Barcelona,Germany
2,24.0,GK,Oliver Baumann,(age 33),0.0,0.0,TSG Hoffenheim,Germany
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Germany
4,2.0,DF,Antonio Rüdiger,(age 31),68.0,3.0,Real Madrid,Germany


In [12]:
import re

def strip_non_numeric(word):
    
    word = re.sub('\D', '', word)
    return int(word)

In [15]:
merged_df = merged_df.dropna(subset=["Player"])
merged_df["Goals"] = merged_df["Goals"].astype(int)
merged_df["Caps"] = merged_df["Caps"].astype(int)
merged_df["No."] = merged_df["No."].fillna(-1).astype(int)
merged_df.rename(columns={"Date of birth (age)": "Age"}, inplace=True)
merged_df["Age"] = merged_df["Age"].apply(strip_non_numeric)

In [16]:
merged_df.head()

,No.,Pos.,Player,Age,Caps,Goals,Club,Team
0,12,GK,Bernd Leno,32,9,0,Fulham,Germany
1,22,GK,Marc-André ter Stegen,31,40,0,Barcelona,Germany
2,24,GK,Oliver Baumann,33,0,0,TSG Hoffenheim,Germany
4,2,DF,Antonio Rüdiger,31,68,3,Real Madrid,Germany
5,3,DF,David Raum,25,20,0,RB Leipzig,Germany


In [17]:
csv_filename = "/euro_player_data.csv"
merged_df.to_csv(DATA_DIR + csv_filename, index=False)

In [18]:
group_letters = ["A", "B", "C", "D", "E", "F"]
general_group_url = "https://en.wikipedia.org/wiki/UEFA_Euro_2024_Group_toreplace"

team_dfs = []

for letter in group_letters:
    group_url = general_group_url.replace("toreplace", letter)
    group_info = pd.read_html(group_url, match="Draw position")
    group_df = group_info[0]
    tmp_df = pd.DataFrame()
    tmp_df["Team"] = group_df["Team"]
    try:
        tmp_df["Playoff_appearences"] = group_df["Finalsappearance"]
    except:
        tmp_df["Playoff_appearences"] = group_df["Finalsappearance[2]"]
    tmp_df["FIFA_rankings"] = group_df["FIFA RankingsJune 2024"]
    tmp_df["Group"] = letter
    team_dfs.append(tmp_df)
    #print(strip_non_numeric(group_df["Finalsappearance"][0]))
    

In [19]:
teams_df = pd.concat(team_dfs, ignore_index=True)

In [20]:
teams_df["Playoff_appearences"] = teams_df["Playoff_appearences"].fillna("0")
teams_df["Playoff_appearences"] = teams_df["Playoff_appearences"].apply(strip_non_numeric)
teams_df["Team"] = teams_df["Team"].str.split("[").str[0]
teams_df.loc[teams_df["Team"] == "Slovakia", "Playoff_appearences"] = 6


In [21]:
csv_filename = "/euro_team_data.csv"
teams_df.to_csv(DATA_DIR + csv_filename, index=False)